In [1]:
import pandas as pd
import numpy as np

od_matrix = pd.read_csv('./data/Sample_OD_Matrix.csv', index_col= 0)
od_matrix.head()

start_location = od_matrix.index[0]
end_location = od_matrix.index[1]
print(start_location)
print(end_location)

orders= pd.read_csv('./workload_res.csv')
orders.head()


oWP_Start
oWP_End


,ORD_NO,SKU_CD,NUM_PCS,LOC,CART_NO,SEQ,PICKER_NO,BATCH_NO
0,ORD_0199,SKU_0075,1,WP_0043,1,1,1.0,8.0
1,ORD_0199,SKU_0069,1,WP_0043,1,2,1.0,8.0
2,ORD_0001,SKU_0246,1,WP_0049,1,3,1.0,8.0
3,ORD_0199,SKU_0246,1,WP_0049,1,4,1.0,8.0
4,ORD_0227,SKU_0254,1,WP_0050,1,5,1.0,8.0


In [2]:
# 경로 전체 합
def calculate_route_distance(route, dist_matrix):
    return sum(dist_matrix.loc[route[i], route[i + 1]] for i in range(len(route) - 1)) # i > 0~10
                                                                                       # route[i], route[i+1] > # 0-1, 1-2, ,,,  10-11 > 경로 전체 함

In [9]:
route = ['oWP_Start', 'WP_0043', 'WP_0049', 'WP_0050', 'WP_0053', 'WP_0063', 'WP_0101', 'WP_0102', 'WP_0103', 'WP_0104', 'WP_0118', 'WP_0142', 'oWP_End']
len(route) # 12

def two_opt(route, od_matrix) :
  best= route
  improved= True
  
  while improved :
    improved = False
    for i in range(1, len(best) - 2) : # i > 1~9 - oWP_Start는 고정
      for j in range(i + 1, len(best) - 1) : # j > 2~10, 3~10, ,,, 10~10 - oWP_End는 고정
        if j - i == 1 : # j-i==1이면 best[i:j]는 원소 1개뿐(바꿀 구간이 없음) > new_route 생성하지 않고 다음 for문(j)
          continue
        new_route = best[:i] + best[i:j][::-1] + best[j:]  # [::-1] 순서 뒤집기 위해서
        if calculate_route_distance(new_route, od_matrix) < calculate_route_distance(best, od_matrix) :
          best = new_route
          improved = True
    if improved :
      break
  return best
  
initial_two_opt_route = two_opt(route, od_matrix)
print(initial_two_opt_route)

# i = 1 / j = 2~10
# i = 1, j= 3
# new_route = best[:1] + best[1:3][::-1] + best[3:]
# 가운데 [1,2]만 바뀜

# i = 1, j= 10
# new_route = best[:1] + best[1:10][::-1] + best[10:]
# 가운데 [2,3,4,,,9]가 역순으로 바뀜

# i = 2, j = 3~10
# ,,,
# i= 8 / j = 9~10

# i = 8, j = 10
# new_route = best[:8] + best[8:10][::-1] + best[10:]
# 가운데 [8,9]만 바뀜

# i = 9 / j= 10
# i= 9, j= 10
# 작업 x

['oWP_Start', 'WP_0053', 'WP_0050', 'WP_0049', 'WP_0063', 'WP_0118', 'WP_0104', 'WP_0103', 'WP_0102', 'WP_0101', 'WP_0043', 'WP_0142', 'oWP_End']


In [11]:
def simulated_annealing(route, dist_matrix, initial_temp=1000, cooling_rate=0.995, min_temp=1):
    current_route = route
    current_distance = calculate_route_distance(current_route, dist_matrix)
    best_route = list(current_route)
    best_distance = current_distance
    temperature = initial_temp
    while temperature > min_temp:
        i, j = sorted(np.random.choice(range(1, len(route) - 1), 2, replace=False))  # exclude start/end - 랜덤으로 LOC 2개 선정
        new_route = current_route[:i] + current_route[i:j][::-1] + current_route[j:]
        new_distance = calculate_route_distance(new_route, dist_matrix)
        # 경로가 좋아졌거나 새 경로가 좋지 않아도 일정 확률로 받아들임
        # 점점 온도가 올라가면서 받아들일 확률이 높아짐
        if new_distance < current_distance or np.random.random() < np.exp((current_distance - new_distance) / temperature):
            current_route = new_route
            current_distance = new_distance
            if new_distance < best_distance:
                best_route = new_route
                best_distance = new_distance
        temperature *= cooling_rate
    return best_route
    
    
simulated_annealing_route = simulated_annealing(initial_two_opt_route, od_matrix, 1000, 0.995, 1)

print(simulated_annealing_route)

['oWP_Start', 'WP_0053', 'WP_0050', 'WP_0049', 'WP_0063', 'WP_0118', 'WP_0104', 'WP_0103', 'WP_0102', 'WP_0101', 'WP_0043', 'WP_0142', 'oWP_End']


In [16]:
# 실제 적용: 카트 단위로 LOC 최적화
orders['SEQ'] = 0
orders.head()

cart_groups = orders.groupby('CART_NO')
cart_groups.head()

# cart_no : 1 cart_df : cart_no==1에 해당하는 orders 
for cart_no, cart_df in cart_groups :     
    locs = cart_df['LOC'].dropna().unique().tolist()
    if len(locs) <= 1:
        route = locs
    else:
        route = [start_location] + locs + [end_location]# 카트별 랙 리스트
        route = two_opt(route, od_matrix)
        route = simulated_annealing(route, od_matrix)
        route = [loc for loc in route if loc != start_location]  # START 제외
        route = [loc for loc in route if loc != end_location]

    seq = 1
    for loc in route:
        mask = (orders['CART_NO'] == cart_no) & (orders['LOC'] == loc)
        loc_indices = orders[mask].index.tolist()
        for idx in loc_indices:
            orders.at[idx, 'SEQ'] = seq
            seq += 1

In [19]:
orders.to_csv('./PRP_SEQ.csv', index= False)

In [ ]:
def solve_picker_routing(self) -> None:
    """피커의 경로를 2-opt + Simulated Annealing 방식으로 최적화"""
    def calculate_total_distance(route, dist_matrix):
        return sum(dist_matrix.loc[route[i], route[i + 1]] for i in range(len(route) - 1))

    def two_opt(route, dist_matrix):
        best = route
        improved = True
        while improved:
            improved = False
            for i in range(1, len(best) - 2):
                for j in range(i + 1, len(best) - 1):  # end 지점은 고정
                    if j - i == 1:
                        continue
                    new_route = best[:i] + best[i:j][::-1] + best[j:]
                    if calculate_total_distance(new_route, dist_matrix) < calculate_total_distance(best, dist_matrix):
                        best = new_route
                        improved = True
            if improved:
                break
        return best

    def simulated_annealing(route, dist_matrix, initial_temp=1000, cooling_rate=0.995, min_temp=1):
        current_route = route
        current_distance = calculate_total_distance(current_route, dist_matrix)
        best_route = list(current_route)
        best_distance = current_distance
        temperature = initial_temp

        while temperature > min_temp:
            i, j = sorted(np.random.choice(range(1, len(route) - 1), 2, replace=False))  # exclude start/end
            new_route = current_route[:i] + current_route[i:j][::-1] + current_route[j:]
            new_distance = calculate_total_distance(new_route, dist_matrix)

            if new_distance < current_distance or np.random.random() < np.exp((current_distance - new_distance) / temperature):
                current_route = new_route
                current_distance = new_distance
                if new_distance < best_distance:
                    best_route = new_route
                    best_distance = new_distance

            temperature *= cooling_rate
        return best_route

    # 실제 적용: 카트 단위로 LOC 최적화
    self.orders['SEQ'] = 0
    cart_groups = self.orders.groupby('CART_NO')
    for cart_no, cart_df in cart_groups:
        locs = cart_df['LOC'].dropna().unique().tolist()
        if len(locs) <= 1:
            route = locs
        else:
            # 전체 경로: START + LOCs + END
            route = [self.start_location] + locs + [self.end_location]
            route = two_opt(route, self.od_matrix)
            route = simulated_annealing(route, self.od_matrix)
            route = [loc for loc in route if loc not in [self.start_location, self.end_location]]  # SEQ는 중간 LOC만

        loc_to_seq = {loc: i+1 for i, loc in enumerate(route)}
        for loc, seq in loc_to_seq.items():
            mask = (self.orders['CART_NO'] == cart_no) & (self.orders['LOC'] == loc)
            self.orders.loc[mask, 'SEQ'] = seq
